# TODO
- vP is not allow to be 0 or 1
- add everything to dictionaries
- attacker only works with dict
- optimise
- try catch when non invertible prime
- t must not be prime --> optimise
- when there is no R large/ small en
- when delta of bounds is small then the possibility of a new generation of p is high 
- when p is smaller than there is is higher chance of success
- Problem when m>=n 
- Use secret module instead of the random module for cryptographic purposes
- camelCase to snake_case if required
- gen_prime() is not used

### Import and Configuration

In [75]:
import sympy
import random
import coloredlogs, logging

In [76]:
coloredlogs.install(fmt='%(levelname)s %(message)s')
logger = logging.getLogger()
logger.setLevel(logging.NOTSET)

In [77]:
MIN_PRIME = 0
MAX_PRIME = 1024
OPT_RANGE_REDUCTION = 24

MAX_ITERS = 10

msg = 12345678900987654321

In [78]:
def gen_prime():
    return sympy.randprime(pow(2,MIN_PRIME), pow(2,MAX_PRIME))

def gen_prime_with_optimized_bit_limit():
    return sympy.randprime(pow(2,MIN_PRIME), pow(2,MAX_PRIME-OPT_RANGE_REDUCTION))

def gen_prime_with_boundaries(lb, ub):
    return sympy.randprime(lb, ub)

def gen_prime_half_size():
    return sympy.randprime(pow(2,MIN_PRIME/2), pow(2,MAX_PRIME/2))

def gen_random_number_with_bit_limit():
    return random.randint(0, pow(2, MAX_PRIME))

def fermat_prime():
    i = random.randint(0,4)
    return pow(2, pow(2, i)) + 1

def modular_multiplicative_inverse(E, phiN):
    return pow(E, -1, phiN)

def calc_n(P, Q):
    return P*Q

def calc_phi_n(P, Q):
    return (P-1) * (Q-1)

def to_binary(dec):
    return "{0:b}".format(dec)

def to_decimal(bin):
    return int(bin, 2)

def concatenate_in_binary(prev, tail):
    prev = to_binary(prev)
    tail = to_binary(tail)

    prev = "0"*(MAX_PRIME-len(prev)) + prev
    tail = "0"*(MAX_PRIME-len(tail)) + tail

    return to_decimal(prev + tail)

def split_in_binary(bin):
    bin = to_binary(bin)
    bin = "0"*(MAX_PRIME*2-len(bin)) + bin
    prev = bin[:len(bin)//2]
    tail = bin[len(bin)//2:]
    return(to_decimal(prev), to_decimal(tail))



# Voraussetzungen

In [79]:
def create_attacker_key():
    a_P = gen_prime_half_size()
    a_Q= gen_prime_half_size()


    a_PhiN = calc_phi_n(a_P, a_Q)
    a_N = calc_n(a_P, a_Q)

    a_E = fermat_prime()
    a_D = modular_multiplicative_inverse(a_E, a_PhiN)

    return {
        "P" : a_P,
        "Q" : a_Q,
        "PhiN" : a_PhiN,
        "N" : a_N,
        "E" : a_E,
        "D" : a_D
    }

attacker = {}

created = False
while not created:
    try:
        attacker = create_attacker_key()
        created = True
    except ValueError:
            logger.error("Trying to generate Key failed. Trying again ...")
logger.info("Attacker Key successfully generated!")

INFO Attacker Key successfully generated!


# Generierung und Verschlüsselung

## Schritt 1, 2 und 3

In [80]:
r_too_large = 0
bounds_too_close = 0

# Do I have a problem when R = 0?
# This has to be optimized
# BAD Inspiration from https://www.cryptool.org/assets/ctp/documents/Kleptografie_kes_4-2010.pdf
user = {}

valid = False
while not valid:
    # Schritt 1
    P = gen_prime_with_optimized_bit_limit()
    vP = pow(P, attacker["E"] ,attacker["N"])

    # Schritt 2
    t = gen_random_number_with_bit_limit() 
    tempN = concatenate_in_binary(vP, t)
    
    # Schritt 3
    # This is wrong, search for Q first. len(Q) ~ len(P). Q not max
    ub = int(int(tempN)//int(P)) 
    lb = int((tempN - pow(2, MAX_PRIME) + 1) // P) 
    logger.info("Bounds: [{0}, {1}] \t P: {2} \t deltaB: {3}".format(lb,ub,P, ub-lb))
    R = 0
    Q = 0
    try:
        for i in range(MAX_ITERS):
            Q = int(gen_prime_with_boundaries(lb, ub))
            R = tempN - P*Q

            if (R < pow(2,MAX_PRIME+1) and R >= 0):
                valid = True
                break
            r_too_large += 1
    except ValueError:
        pass
    user["P"] = P
    user["Q"] = Q

attacker["vP"] = vP
attacker["R"] = R

logger.info("Amount of failed attempts: {0}".format(r_too_large))

logger.info("tempN = P * Q + R")
logger.info("{0} = {1} * {2} + {3}".format(tempN, P, Q, R))

INFO Bounds: [5327821851947124912854804767860301894030521826172964347992769845765050440387905495733022945938721471708072913167831546873140372244889596258897269269651689995329771369373528248399818904742261175295227906961912696586491549916708202338932095802384476141692710583172612937272491640329407910681180989919372900704319368512, 5327821851947124912854804767860301894030521826172964347992769845765050440387905495733022945938721471708072913167831546873140372244889596258897269269651689995329771369373528248399818904742261175295227906961912696586491549916708202338932095802384476141692710583172612937272491640329407910681180989919372900704699198348] 	 P: 473289078569884861930043354166795230562462197246795955016983443660409280257944787802817327494159604203199539334714328452427426544882247987769178491803729909485508182905587332007429717397075769382782192923180387720114256944626299427373532373832298603610436435645040512642331337491594104649669715893511 	 deltaB: 379829836
INFO Amount of failed at

## Schritt 4

In [81]:
user["N"] = calc_n(user["P"], user["Q"])

## Schritt 5

In [82]:
def create_user_key():
    PhiN = calc_phi_n(P, Q)
    E = fermat_prime()
    D = modular_multiplicative_inverse(E, PhiN)

    return  {
        "PhiN" : PhiN,
        "E" : E,
        "D" : D
    }

created = False
while not created:
    try:
        user.update(create_user_key())
        created = True
    except ValueError:
            logger.info("Trying to generate Key failed. Trying again ...")
logger.info("User Key successfully generated!")

INFO User Key successfully generated!


## Schritt 6

In [83]:
cipher = pow(msg, user["E"], user["N"])
signature = pow(msg, user["D"], user["N"])

# Angriff

## Schritt 1

In [84]:
pub_user = {
    "N" : user["N"],
    "E" : user["E"]
}

collection_user = {
    "N" : pub_user["N"],
    "E" : pub_user["E"]
}

## Schritt 2

In [85]:
collection_user["vP"] = split_in_binary(user["N"])[0]
collection_user["vP'"] = collection_user["vP"]+1

### Validation test for vP

In [86]:
testVar = "vP"
altTestVar = "vP'"

if(collection_user[testVar] == attacker[testVar] or collection_user[altTestVar] == attacker[testVar]):
    logger.info("Success!!!")
    logger.info("The variable {0} or {1} is in the expected state: {2}".format(testVar, altTestVar, attacker[testVar]))
    logger.info("Current {0}: {1} \t Current {2}: {3}".format(testVar, collection_user[testVar], altTestVar, collection_user[altTestVar]))
else:
    logger.error("Error!!!")
    logger.error("The variable {0} doesn't have the expected value.".format(testVar))
    logger.error("Expected: {0} \t Current: {1}".format(attacker[testVar], collection_user[testVar]))

INFO Success!!!
INFO The variable vP or vP' is in the expected state: 14026865020462348781318720497973575900107645421803768565624944261068875224356446576173547640380990164890378387785595275210420581942435531127000730713616787522020140384205953100708152671529878401102435693288953172626254084699061710359614424336257465820253875671109545468968548635961948141518058753558895740252
INFO Current vP: 14026865020462348781318720497973575900107645421803768565624944261068875224356446576173547640380990164890378387785595275210420581942435531127000730713616787522020140384205953100708152671529878401102435693288953172626254084699061710359614424336257465820253875671109545468968548635961948141518058753558895740252 	 Current vP': 1402686502046234878131872049797357590010764542180376856562494426106887522435644657617354764038099016489037838778559527521042058194243553112700073071361678752202014038420595310070815267152987840110243569328895317262625408469906171035961442433625746582025387567110954546896854863596

## Schritt 3

In [87]:
collection_user["P"] = pow(collection_user["vP"], attacker["D"], attacker["N"])
collection_user["P'"] = pow(collection_user["vP'"], attacker["D"], attacker["N"])

### Validation test for P and P'

In [88]:
testVar = "P"
altTestVar = "P'"

if(collection_user[testVar] == user[testVar] or collection_user[altTestVar] == user[testVar]):
    logger.info("Success!!!")
    logger.info("The variables {0} or {1} are in the expected state: {2}".format(testVar, altTestVar, user["P"]))
    logger.info("Current {0}: {1} \t Current {2}: {3}".format(testVar, collection_user[testVar], altTestVar, collection_user[altTestVar]))
else:
    logger.error("Error!!!")
    logger.error("The variable {0} or {1} doesn't have the expected value.".format(testVar, altTestVar))
    logger.error("Expected: {0} \t Current: {1} or {2}".format(user[testVar], collection_user[testVar], collection_user[altTestVar]))

INFO Success!!!
INFO The variables P or P' are in the expected state: 473289078569884861930043354166795230562462197246795955016983443660409280257944787802817327494159604203199539334714328452427426544882247987769178491803729909485508182905587332007429717397075769382782192923180387720114256944626299427373532373832298603610436435645040512642331337491594104649669715893511
INFO Current P: 473289078569884861930043354166795230562462197246795955016983443660409280257944787802817327494159604203199539334714328452427426544882247987769178491803729909485508182905587332007429717397075769382782192923180387720114256944626299427373532373832298603610436435645040512642331337491594104649669715893511 	 Current P': 1104881230654675912858404658846592014606626290283346645087978965838698732560838609563849532045342765650985539848130357587503584598523782481676543351472871399026067451325763726818287898193673774472070115478177386117173928941654619603998531115224042125422296772428695569941641870069558627229839376274

# Schritt 4

In [89]:
# Potential to optimize when N/P not hole Number
collection_user["Q"] = int(pub_user["N"]//collection_user["P"])
collection_user["Q'"] = int(pub_user["N"]//collection_user["P'"])

### Validation test for Q and Q'

In [90]:
testVar = "Q"
altTestVar = "Q'"

if(collection_user[testVar] == user[testVar] or collection_user[altTestVar] == user[testVar]):
    logger.info("Success!!!")
    logger.info("The variables {0} or {1} are in the expected state!".format(testVar, altTestVar))
    logger.info("Current {0}: {1} \t Current {2}: {3}".format(testVar, collection_user[testVar], altTestVar, collection_user[altTestVar]))
else:
    logger.error("Error!!!")
    logger.error("The variable {0} doesn't have the expected value.".format(testVar))
    logger.error("Expected: {0} \t Current: {1} or {2}".format(attacker[testVar], collection_user[testVar], collection_user[altTestVar]))

INFO Success!!!
INFO The variables Q or Q' are in the expected state!
INFO Current Q: 5327821851947124912854804767860301894030521826172964347992769845765050440387905495733022945938721471708072913167831546873140372244889596258897269269651689995329771369373528248399818904742261175295227906961912696586491549916708202338932095802384476141692710583172612937272491640329407910681180989919372900704635396449 	 Current Q': 228223615817822087087754301374368753990425422325609481870215567206078030586265845078655400993343375114943164509802738223191067178470191810449724968685504264466731237365121713533836264352222080548143894067462567368249574268406638897418813094500802004313701744075507471568692371369321571274813676985245649020211


## Schritt 5

In [91]:
collection_user["PhiN"] = int(calc_phi_n(collection_user["P"], collection_user["Q"]))
collection_user["PhiN'"] = int(calc_phi_n(collection_user["P'"], collection_user["Q'"]))

collection_user["D"] = modular_multiplicative_inverse(pub_user["E"], collection_user["PhiN"])
collection_user["D'"] = modular_multiplicative_inverse(pub_user["E"], collection_user["PhiN'"])

### Validation test for PhiN and PhiN'

In [92]:
testVar = "PhiN"
altTestVar = "PhiN'"

if(collection_user[testVar] == user[testVar] or collection_user[altTestVar] == user[testVar]):
    logger.info("Success!!!")
    logger.info("The variables {0} or {1} are in the expected state!".format(testVar, altTestVar))
    logger.info("Current {0}: {1} \t Current {2}: {3}".format(testVar, collection_user[testVar], altTestVar, collection_user[altTestVar]))
else:
    logger.error("Error!!!")
    logger.error("The variable {0} doesn't have the expected value.".format(testVar))
    logger.error("Expected: {0} \t Current: {1} or {2}".format(attacker[testVar], collection_user[testVar], collection_user[altTestVar]))

INFO Success!!!
INFO The variables PhiN or PhiN' are in the expected state!
INFO Current PhiN: 2521599895092552275240636062350554603341250165645906408720860996614838602440257193418281430402658211343739241820245273157031471959364397936018793461043796754927147815230559931909415121777165432563292941916904115444097737387001224968708140851235668878131460475999103714515340821884724917120087615715750999488357541571067938022620174344885094215926213900207218632420712120916929527295716802301896983034890567637034764427797020825017433098291824866847677374492479091208202584578021610722436708832347267475327750111125389967043229488399144408417889764814664031166335706726424692282322894823989021902000289882945400252480 	 Current PhiN': 2521599895092552275240636062350554603341250165645906408720860996614838602440257193418281430402658211343739241820245273157031471959364397936018793461043796754927147815230559931909415121777165432563292941916904115444097737387001224968708140851235668878131460475999103714515

### Validation test for D and D'

In [93]:
testVar = "D"
altTestVar = "D'"

if(collection_user[testVar] == user[testVar] or collection_user[altTestVar] == user[testVar]):
    logger.info("Success!!!")
    logger.info("The variables {0} or {1} are in the expected state!".format(testVar, altTestVar))
    logger.info("Current {0}: {1} \t Current {2}: {3}".format(testVar, collection_user[testVar], altTestVar, collection_user[altTestVar]))
else:
    logger.error("Error!!!")
    logger.error("The variable {0} doesn't have the expected value.".format(testVar))
    logger.error("Expected: {0} \t Current: {1} or {2}".format(attacker[testVar], collection_user[testVar], collection_user[altTestVar]))

INFO Success!!!
INFO The variables D or D' are in the expected state!
INFO Current D: 51788660738126178532338925186136785267823103330323176619898361253087153194143555279323234284785357164177075842501947566555752647470962656543346445497428176940231334350066888450036967726811795966098422262541052220696027503897090938063706876814062442741732850156320759261755172593448582303792329993032956121142701847119298176273658462673227898967555486361580729652536407140305280127867232493070377636525362833810653415930158384278704624110056857512198815113094539828894222787769002060399466104465255077617088845225975783078874328873540875745464083242146234736864486645006143641179892525338194050385162430114965725313 	 Current D': 333779072736437294775661349175138642049305662251525673980399913722534215422879154567703608781956146656193263695174142005847811453796880420143781883127926771884477582085312226823232314286844227344653650169051729579894530903645812695172850784815744198205447604090700287218221487554358433

## Schritt 6

In [94]:
collection_user["Sig"] = pow(msg, collection_user["D"], user["N"])
collection_user["Sig'"] = pow(msg, collection_user["D'"], user["N"])

### Validation test for signature and signature'

In [95]:
testVar = "Sig"
altTestVar = "Sig'"

if(collection_user[testVar] == signature):
    collection_user["correct_key"] = "D"
    logger.info("D is the right privat Key: {0}".format(collection_user[testVar]))
elif(collection_user[altTestVar] == signature):
    collection_user["correct_key"] = "D'"
    logger.info("D' is the right privat Key: {0}".format(collection_user[altTestVar]))
else:
    logger.error("Message was incorrectly signed!")

INFO D is the right privat Key: 2295281130139547790540513907408043651015202671443212773098875507466381092315474845684448906169822263803156538605256383082049802333584049252663744255677101671036912611345748518456643319577958402835559895144346763279333357172766937344348171126117714588165045546181437238668266288066516991605183023061842431615309084526740684039911377758510039942232950507793067939079641913824214207415073720516173742606062851496395226565002800458313137184876841506978614108447967574195873268614248088762266766579664426438361297421138405262311426281417573566832697506968972226207011244130488385151930878961689387900177274091821866954013


# Result

In [96]:
def sort_dic(dict):
    return {key: dict[key] for key in sorted(dict)}

In [97]:
logger.info(sort_dic(attacker))
logger.info(sort_dic(user))
logger.info(sort_dic(collection_user))



INFO {'D': 2495892479654988337727641166200997649325762472709709796615513651349558602548450336691487304714641170942663036222055167650595084596691798404257627146848941432397360823051238361409194615582811736117055136928700219816514613109644299615065588050008430211496875043507763853743924621172592736096400190447676333617, 'E': 65537, 'N': 15614099411908072803518176700010956752946019012407049536157972333762506885759621011430889985594066286757284021084844360664094125545378998856417727216784955433909107618695113310403862970031332635725132488463120284813209706042887094678546874184955921180622935796928692090768416663836336851906502903607990391313, 'P': 1609887530256126368750103544404013288334265900427726583693861238708363282060142660567142185705349854798194863634057442471786770025854839387393127443802199, 'PhiN': 1561409941190807280351817670001095675294601901240704953615797233376250688575962101143088998559406628675728402108484436066409412554537899885641772721678494412514565065485958462119839514